In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import display
from sklearn.preprocessing import Normalizer
from scipy.stats import variation
import random
import imblearn
from imblearn.over_sampling import SMOTENC
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.impute import KNNImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

## Imputation MICE et KNN

In [ ]:

# 1) Assemblage du dataset à impute
CES19_data = pd.read_csv('C:/Users/nnass/Documents/data_GLO-7027/CES19.csv')

CES19_converted = CES19_data.infer_objects()

data_ces_only = CES19_converted.copy()
data_ces_only = data_ces_only.loc[:,~data_ces_only.columns.str.startswith('pes19')].copy()

list_todel = ['cps19_votechoice', 'cps19_votechoice_pr', 'cps19_vote_unlikely', 'cps19_vote_unlike_pr', 'cps19_v_advance',
              'cps19_votechoice_7_TEXT', 'cps19_votechoice_pr_7_TEXT', 'cps19_vote_unlikely_7_TEXT', 'cps19_vote_unlike_pr_7_TEXT',
              'cps19_v_advance_7_TEXT', 'cps19_vote_lean', 'cps19_vote_lean_7_TEXT', 'cps19_vote_lean_pr', 'cps19_vote_lean_pr_7_TEXT',
              'cps19_2nd_choice', 'cps19_2nd_choice_7_TEXT', 'cps19_2nd_choice_pr', 'cps19_2nd_choice_pr_7_TEXT', 'cps19_not_vote_for_1',
              'cps19_not_vote_for_2', 'cps19_not_vote_for_3', 'cps19_not_vote_for_4', 'cps19_not_vote_for_5', 'cps19_not_vote_for_6',
              'cps19_not_vote_for_7', 'cps19_not_vote_for_8', 'cps19_not_vote_for_9', 'cps19_not_vote_for_7_TEXT']

# Une variété de colonnes de metadata sont aussi retirées
list_meta = ['cps19_weight_general_all', 'cps19_weight_general_restricted',
             'cps19_duplicates_flag', 'get_news', 'get_more_naming',
             'get_not_vote_for', 'get_party_issue_handling', 'get_imp_loc_iss',
             'get_outcome', 'justice_law', 'justice_law_fr', 'lr_scale_order',
             'ethnicity_intro', 'ethnicity_intro_fr', 'premier', 'province_fr',
             'pid_en', 'pid_party_en', 'pid_party_fr', 'confidence_institutions_word',
             'confidence_institutions_word_fr', 'govt_programs_word',
             'govt_programs_word_fr', 'split_taxes', 'split_senate', 'split_trade',
             'split_lifesat', 'split_responsibility', 'split_sexism',
             'split_abortion', 'split_getahead', 'split_att_div', 'split_govt_eff',
             'split_medical', 'split_ties', 'split_health_followups',
             'split_gender_id', 'split_big5', 'split_hatespeech', 'split_vol_assoc',
             'notvote_split', 'splitsample', 'constituencyname', 'pes10_socnet3']
             # La dernière colonne est une question du pes

# Puis, les cinq colonnes d'intentions de vote sont combinées en une seule (qui gardera le nom vote_choice):
votes = data_ces_only[["cps19_votechoice", "cps19_votechoice_pr", "cps19_vote_unlikely", "cps19_vote_unlike_pr", "cps19_v_advance"]]
votes = votes.fillna('').sum(axis=1)
data_ces_only['cps19_votechoice'] = votes

# Certaines rangées ne contenaient aucune intention de vote. La chaîne vide est
# replacée par un NaN
data_ces_only['cps19_votechoice'] = np.where(data_ces_only['cps19_votechoice'] == '', np.NaN,
                                             data_ces_only['cps19_votechoice'])

# Les intentions de vote que l'on cherche à prédire sont aussi importées, de façon à pouvoir être retirées
to_pred = pd.read_table('C:/Users/nnass/Documents/data_GLO-7027/exemple_GLO.txt',
                        header=None, index_col=0)
# On se retrouve bien avec un NaN en intention de vote pour chacune de ces rangées, donc l'étape précédente semble
# être correcte

# Les colonnes inutiles sont retirées et les intentions de vote sont préalablement séparées
data_todel = list_todel + list_meta
vote_int = data_ces_only[list_todel].copy()
data_ready = data_ces_only.drop(labels=data_todel, axis=1)
# Les IDs et les dates sont retirés pour l'instant
data_ready = data_ready.drop(labels=['Unnamed: 0', 'cps19_consent',
                                     'cps19_StartDate', 'cps19_EndDate',
                                     'cps19_ResponseId'], axis=1)

# Puis, les données numériques et textuelles sont séparées
num_cols = np.where(data_ready.dtypes != 'object')
data_num = data_ready.iloc[:, np.r_[num_cols]]

text_cols = np.where(data_ready.dtypes == 'object')
data_text = data_ready.iloc[:, np.r_[text_cols]]

# Des données numériques qui ne sont pas utiles sont retirées
num_not = ['cps19_current_date', 'cps19_current_date_string',
           'cps19_Q_TotalDuration', 'cps19_data_quality', 'cps19_inattentive',
           'constituencynumber']
data_num = data_num.drop(labels=num_not, axis=1)

# Dataframe de one-hot vectors pour les attributs encodés ainsi:
data_text_NaN = data_text.fillna('NaN')
uniques = data_text_NaN.nunique()

one_hot = np.where(uniques == 2)
one_hot_df = data_text_NaN.iloc[:, np.r_[one_hot]]

one_hot_df = one_hot_df.drop(labels=['cps19_Q_Language'], axis=1)
one_hot_df.iloc[:, :] = np.where(one_hot_df.iloc[:, :] == 'NaN', 0, 1)

# On peut ensuite assembler le dataset utilisé pour l'imputation. Afin d'accélérer le processus et de limiter le bruit,
# seuls les attributs qui seront utilisés dans au moins une tentative de modèle sont conservés
num_impute = data_num[['cps19_interest_gen_1', 'cps19_interest_elxn_1', 'cps19_age', 'cps19_party_rating_23',
                       'cps19_party_rating_24', 'cps19_party_rating_25', 'cps19_party_rating_26',
                       'cps19_party_rating_27', 'cps19_party_rating_28', 'cps19_lead_rating_23',
                       'cps19_lead_rating_24', 'cps19_lead_rating_25', 'cps19_lead_rating_26',
                       'cps19_lead_rating_27', 'cps19_lead_rating_28', 'cps19_cand_rating_23',
                       'cps19_cand_rating_24', 'cps19_cand_rating_25',
                       'cps19_cand_rating_27', 'cps19_cand_rating_28']].copy()

cats_tokeep = data_text[['cps19_imp_iss_party', 'cps19_fed_id', 'cps19_vote_2015', 'cps19_prov_id', 'cps19_fed_gov_sat',
                         'cps19_issue_handle_8', 'cps19_issue_handle_2']].copy()

cats_tokeep = pd.get_dummies(cats_tokeep)

mask_onehot = one_hot_df.columns.str.contains(r'cps19_lead_*')
one_hot_sel = one_hot_df.loc[:, mask_onehot]
one_hot_keep = one_hot_df[['cps19_ethnicity_38', 'cps19_language_69', 'cps19_party_member_38']]
one_hot_keep = pd.concat([one_hot_keep, one_hot_sel], axis=1)

# Dataset final
data_selected = pd.concat([one_hot_keep, num_impute, cats_tokeep], axis=1)

# Les données numériques sont normalisées min-max
scaler = MinMaxScaler()
data_selected = pd.DataFrame(scaler.fit_transform(data_selected), columns=data_selected.columns,
                             index=data_selected.index)

# Toutes les intentions de vote sont ajoutées et le dataset est subdivisé
data_selected = pd.concat([data_selected, vote_int['cps19_votechoice']], axis=1)

features_data = data_selected.drop(labels=to_pred.index, axis=0)
features_data = features_data.dropna(subset='cps19_votechoice')
labels_train = features_data['cps19_votechoice']
features_data = features_data.drop(labels='cps19_votechoice', axis=1)

to_pred_feats = data_selected.iloc[np.r_[to_pred.index], :]
to_pred_feats = data_selected.drop(labels='cps19_votechoice', axis=1)

# L'ensemble de données "d'entraînement" est divisé en un ensemble de training (75%) et de test (25%) afin de pouvoir
# évaluer la justesse de l'imputation des données manquantes
train_feats, test_feats, train_labels, test_labels = train_test_split(features_data, labels_train,
                                                                      test_size=0.25, random_state=42)

# Les ensembles de données sont sauvegardés
train_feats.to_csv('C:/Users/nnass/Documents/data_GLO-7027/train_feats_impute.csv')
train_labels.to_csv('C:/Users/nnass/Documents/data_GLO-7027/train_labels_impute.csv')

test_feats.to_csv('C:/Users/nnass/Documents/data_GLO-7027/test_feats_impute.csv')
test_labels.to_csv('C:/Users/nnass/Documents/data_GLO-7027/test_labels_impute.csv')

to_pred_feats.to_csv('C:/Users/nnass/Documents/data_GLO-7027/to_pred_selected.csv')

# 2) Imputation par MICE
# D'abord, l'imputer est fit sur l'ensemble train_feats
lr = LinearRegression()
imp = IterativeImputer(estimator=lr, missing_values=np.nan, max_iter=40, verbose=2, imputation_order='roman',
                       random_state=0)

train_feats_fit = imp.fit_transform(train_feats)
train_feats_fit = pd.DataFrame(train_feats_fit, columns=train_feats.columns, index=train_feats.index)

train_feats_fit.to_csv('C:/Users/nnass/Documents/data_GLO-7027/train_feats_MICE.csv')

# Ensuite, il est utilisé pour transformer test_feats
test_feats_MICE = imp.transform(test_feats)
test_feats_MICE = pd.DataFrame(test_feats_MICE, columns=test_feats.columns, index=test_feats.index)

test_feats_MICE.to_csv('C:/Users/nnass/Documents/data_GLO-7027/test_feats_MICE.csv')

# Finalement, les données pour lesquelles on doit faire la prédiction sont aussi transformées
to_pred_feats_MICE = imp.transform(to_pred_feats)
to_pred_feats_MICE = pd.DataFrame(to_pred_feats_MICE, columns=to_pred_feats.columns, index=to_pred_feats.index)

to_pred_feats_MICE.to_csv('C:/Users/nnass/Documents/data_GLO-7027/topred_feats_MICE.csv')

# 3) Imputation par KNN
# Une boucle est d'abord nécessaire pour identifier le paramètre k optimal
df_model = pd.DataFrame(columns=['k', 'f1_weighted', 'balanced_accuracy'])
results = df_model.copy()
strategies = [5, 10, 15, 20, 25, 35, 50, 75, 100, 150, 500]  # k tentés

# Paramètres
model = DecisionTreeClassifier(class_weight='balanced')
cv = KFold(n_splits=10)
scoring = ['f1_macro', 'balanced_accuracy']

# Scaling des données numériques
scaler = MinMaxScaler()
train_feats = pd.DataFrame(scaler.fit_transform(train_feats), columns=train_feats.columns)

for k in strategies:
    # Création du pipeline
    pipeline = Pipeline(steps=[('KNN', KNNImputer(n_neighbors=k)),
                               ('Tree', model)])

    # Évaluation
    scores = cross_validate(pipeline, train_feats, train_labels, scoring=scoring,
                            cv=cv, error_score='raise', verbose=2)

    F1 = scores['test_f1_macro']
    accuracy = scores['test_balanced_accuracy']

    # Sauvegarde des résultats
    res_CV = df_model.copy()
    res_CV['f1_macro'] = F1
    res_CV['balanced_accuracy'] = accuracy
    res_CV['k'] = k

    results = pd.concat([results, res_CV]).reset_index(drop=True)
# Figure pour représenter les résultats
fig, axs = plt.subplots(2, 1, figsize=(12, 8))

med_f1 = results[results['k'] == 50]['f1_macro'].median()
med_accuracy = results[results['k'] == 50]['balanced_accuracy'].median()

sns.boxplot(x='k', y='f1_macro', data=results, ax=axs[0],
            palette='plasma')
axs[0].axhline(y=med_f1, c='k', linestyle='--', linewidth=2.5)

sns.boxplot(x='k', y='balanced_accuracy', data=results, ax=axs[1],
            palette='plasma')
axs[1].axhline(y=med_accuracy, c='k', linestyle='--', linewidth=2.5)

# Axes
axs[0].set_xlabel('')
axs[1].set_xlabel('Nombre de voisins k', fontsize=14)

axs[0].set_ylabel('Score F1 macro', fontsize=14)
axs[1].set_ylabel('Exactitude équilibrée', fontsize=14)

Fig_k = plt.gcf()
Fig_k.savefig('C:/Users/nnass/Documents/data_GLO-7027/Figure_KNN_k.pdf', bbox_inches='tight')

# Un k de 50 semble donner les meilleurs résultats (à confirmer avec F1-macro)
# On l'utilise donc pour l'imputation

# D'abord sur le set "d'entraînement"
imputer = KNNImputer(n_neighbors=50)
train_feats_KNN = imputer.fit_transform(train_feats)
train_feats_KNN = pd.DataFrame(train_feats_KNN, columns=train_feats.columns, index=train_feats.index)

train_feats_KNN.to_csv('C:/Users/nnass/Documents/data_GLO-7027/train_feats_KNN.csv')

# Puis pour le set de test
test_feats_KNN = imputer.transform(test_feats)
test_feats_KNN = pd.DataFrame(test_feats_KNN, columns=test_feats.columns, index=test_feats.index)

test_feats_KNN.to_csv('C:/Users/nnass/Documents/data_GLO-7027/test_feats_KNN.csv')

# Et finalement, pour les données sur lesquelles on doit faire nos prédictions
to_pred_feats_KNN = imputer.transform(to_pred_feats)
to_pred_feats_KNN = pd.DataFrame(to_pred_feats_KNN, columns=to_pred_feats.columns, index=to_pred_feats.index)

to_pred_feats_KNN.to_csv('C:/Users/nnass/Documents/data_GLO-7027/topred_feats_KNN.csv')
